In [6]:
a=[0,1,2,3,4,5]
a[:3]

[0, 1, 2]

In [133]:
class Node:
    def __init__(self, isleaf=False):
        self.isleaf=isleaf
        self.keys=[]
        self.child=[]
    def __repr__(self):
        return f'k:{self.keys}'
    def split(self):
        length = len(self.keys)
        temp = Node(self.isleaf)
        mid = self.keys[length//2]
        temp.keys = self.keys[length//2+1:]
        self.keys = self.keys[:length//2]
        if self.child:
            temp.child = self.child[length//2+1:]
            self.child = self.child[:length//2+1]
        return self,mid,temp
        
# nn=Node()
# nn.keys=[10,20,30,40,50]
# nn.child=[[1,2],[12,13],[22,23],[33,39],[45,49],[55,60]]
# ll,mid,rr=nn.split()
# ll,ll.child , rr, rr.child

In [231]:
import random

class BTree:
    def __init__(self, m):
        self.m=m #차수 -> m개의 child를 가질 수 있음
        self.root=Node(True)
        # self.root.isleaf = 'root'
        # self.root=None

    def printTree(self, node ,level=0):
        print(f'level {level} , length : {len(node.keys)} ', end=" : ")
        for i in node.keys: 
            print(i[0],end=" ")
        level+=1
        if len(node.child)>0:
            for i in node.child: 
                self.printTree(i, level)
        
    def splitNode(self,p_pos,p_node,node):
        pass

    # k : Key 
    # def insert_node(self,p_pos,p_node,node,key):
    def insert_node(self,node,key):
        """ 데이터 추가하는것, Root부터 시작해서 내려가도록 Trigger가 발동되며, 
        현재의 노드(node)가 leaf가 아닐경우, 재귀적으로 child를 내려감. 
        리프에서의 삽입이 이뤄지고나면, 그대로 자기자신만 반환하고, 
        Leaf에서의 삽입이 OverFlow가 일어날경우 중간key와 자기자신을 반환
        삽입을 하고난 이후, 반환되는 key가 있다면, 해당 child에서 OverFlow가 난것이므로, 관련연산 진행
        """
        pos=0
        while pos < len(node.keys):
            # print(pos)
            if key[0]==node.keys[pos][0]:
                #Duplicate
                print(f"Duplicates key {key}")
                return node, None 
            elif key[0]<node.keys[pos][0]: break
            pos+=1
        print(key," Node : ",node , pos)
        
        # Position 찾기 완료
        if node.isleaf==True : # Leaf라면?? -> Leaf에서의 삽입 로직
            i=len(node.keys)-1
            node.keys.append([1e9,None])
            while i>=0 and key[0]<node.keys[i][0]:
                node.keys[i+1]=node.keys[i]
                i-=1
            node.keys[i+1]=key
            if len(node.keys) >= self.m : # 오버플로 발생?
                if node == self.root:
                    # print("Root OverFlow")
                    ll,mid,rr=node.split()
                    self.root=Node(False)
                    self.root.keys.append(mid)
                    self.root.child.append(ll)
                    self.root.child.append(rr)
                    return self.root, None
                else:
                    # print("Leaf OVERFLOW",node)
                    return node, node.keys[(self.m//2)]

            else: return node, None
        # elif node.isleaf == False: #Leaf가 아니라면?
        else: #Leaf가 아니라면?
            node.child[pos], child_mid_kv = self.insert_node(node.child[pos],key)
            if child_mid_kv: # Child에서 OverFlow발생 -> 자기에 추가시켜줘야해
                ll,mid,rr=node.child[pos].split()

                i=len(node.keys)-1
                node.keys.append([1e9,None])
                node.child.append([])
                while i>=0 and mid[0] < node.keys[i][0]:
                    node.keys[i+1]=node.keys[i]
                    node.child[i+2]=node.child[i+1]
                    i-=1
                node.keys[i+1]=mid
                # i=len(node.child)-1
                node.child[i+2]=rr
                if len(node.keys) >= self.m : # 오버플로 발생?
                    if node == self.root:
                        # print("Root OverFlow")
                        ll,mid,rr=node.split()
                        self.root=Node(False)
                        self.root.keys.append(mid)
                        self.root.child.append(ll)
                        self.root.child.append(rr)
                        return self.root, None
                if len(node.keys)>=self.m:
                    return node, node.keys[(self.m//2)]
            return node, None
            




            #     # Parent로 올라가
            #     temp=node.keys[pos:]
            #     node.keys[pos+1:] = temp
            #     # node.keys[pos] = node.child.keys[child_mid_kv]
            #     node.keys[pos] = child_mid_kv
            #     node.child[pos+2:] = node.child[pos+1:]
                
            #     # child split
            #     c_node=node.child[pos]
            #     temp = Node(c_node.isleaf) # 오른쪽으로 추가될 노드
            #     temp.keys = c_node.keys[self.m//2+1:] ## m//2는 부모로 올라갔으므로
            #     c_node.keys=c_node.keys[:self.m//2]
                
            #     ## Child 의 child까지 정리
            #     node.child[pos+1] = temp
            #     if not c_node.isleaf:
            #         temp.child = c_node.child[self.m//2+1:]
            #         c_node.child = c_node.child[:self.m//2+1]
            
            # if len(node.keys) >= self.m : # 오버플로 발생?
            #     print('overflow check')
            #     return node, node.keys[(self.m//2)]
            # else:# Child Overflow X 
            #     return node , None
            
            # 자기자신 OverFlow?
            # if len(node.keys) == self.m:
            #     node = splitNode(p_pos,p_node,node)
                
                

        

# if __name__=="__main__":
print("Start")
# kvs = [[1,11],[7,77],[5,55],[6,66],[4,44],[3,33],[2,22],[23,9]]

bt = BTree(3)
bt.insert_node(bt.root,[10,11])
bt.insert_node(bt.root,[15,11])
bt.insert_node(bt.root,[18,11])
bt.insert_node(bt.root,[30,11])
bt.insert_node(bt.root,[20,11])  
bt.insert_node(bt.root,[40,11])
bt.insert_node(bt.root,[50,11])
bt.insert_node(bt.root,[60,11])
bt.insert_node(bt.root,[70,11])
bt.insert_node(bt.root,[80,11])
bt.insert_node(bt.root,[90,11])
bt.insert_node(bt.root,[85,11])
bt.insert_node(bt.root,[75,11])
bt.insert_node(bt.root,[77,11])
bt.root

Start
[10, 11]  Node :  k:[] 0
[15, 11]  Node :  k:[[10, 11]] 1
[18, 11]  Node :  k:[[10, 11], [15, 11]] 2
[30, 11]  Node :  k:[[15, 11]] 1
[30, 11]  Node :  k:[[18, 11]] 1
[20, 11]  Node :  k:[[15, 11]] 1
[20, 11]  Node :  k:[[18, 11], [30, 11]] 1
[40, 11]  Node :  k:[[15, 11], [20, 11]] 2
[40, 11]  Node :  k:[[30, 11]] 1
[50, 11]  Node :  k:[[15, 11], [20, 11]] 2
[50, 11]  Node :  k:[[30, 11], [40, 11]] 2
[60, 11]  Node :  k:[[20, 11]] 1
[60, 11]  Node :  k:[[40, 11]] 1
[60, 11]  Node :  k:[[50, 11]] 1
[70, 11]  Node :  k:[[20, 11]] 1
[70, 11]  Node :  k:[[40, 11]] 1
[70, 11]  Node :  k:[[50, 11], [60, 11]] 2
[80, 11]  Node :  k:[[20, 11]] 1
[80, 11]  Node :  k:[[40, 11], [60, 11]] 2
[80, 11]  Node :  k:[[70, 11]] 1
[90, 11]  Node :  k:[[20, 11]] 1
[90, 11]  Node :  k:[[40, 11], [60, 11]] 2
[90, 11]  Node :  k:[[70, 11], [80, 11]] 2
[85, 11]  Node :  k:[[20, 11], [60, 11]] 2
[85, 11]  Node :  k:[[80, 11]] 1
[85, 11]  Node :  k:[[90, 11]] 0
[75, 11]  Node :  k:[[20, 11], [60, 11]] 2
[

k:[[20, 11], [60, 11]]

In [232]:
bt.root,'---', bt.root.child

(k:[[20, 11], [60, 11]],
 '---',
 [k:[[15, 11]], k:[[40, 11]], k:[[75, 11], [80, 11]]])

In [233]:
bt.root.child[0].child , bt.root.child[1].child

([k:[[10, 11]], k:[[18, 11]]], [k:[[30, 11]], k:[[50, 11]]])

In [251]:
# bt.root.child[0].child[1].child

[]